In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h3>Họ tên: Phương Anh Mỹ</h3>
<h3>MSSV: 18020918</h3>

**1. Định nghĩa bài toán**
<p>Một vấn đề tồn tại đối với bất kỳ trang web lớn nào hiện nay là làm thế nào để xử lý nội dung độc hại và gây chia rẽ. Quora muốn giải quyết vấn đề này trực tiếp để giữ cho nền tảng của họ trở thành một nơi mà người dùng có thể cảm thấy an toàn khi chia sẻ kiến thức của họ với cộng đồng.</p>
<p>Quora là một nền tảng cho phép mọi người học hỏi và chia sẻ tri thức. Tại Quora, mọi người có thể đặt câu hỏi và kết nối với những người khác. Một thách thức lớn, đồng thời cũng là mục tiêu của bài toán, là xác định được những câu hỏi có nội dung nhạy cảm để có thể loại bỏ chúng.</p>
<p>Input: Câu hỏi từ Quora</p>
<p>Output: giá trị 0 hoặc 1 (0: câu hỏi không phản cảm; 1: câu hỏi phản cảm)</p>


**2. Dữ liệu**

2.1. Khảo sát dữ liệu

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [ ]:
test_df = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')
train_df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')

<p>Dữ liệu gồm 3 cột:</p>
<p>qid: mã số câu hỏi</p>
<p>question_text: câu hỏi</p> 
<p>target: nhãn dữ liệu (0 hoặc 1)</p>

In [ ]:
train_df.info()

In [ ]:
test_df.info()

Tập train gồm có 1306122 câu hỏi, tập test gồm có 375806 câu hỏi

In [ ]:
sincere_question = train_df[train_df['target'] == 0].question_text #những câu hỏi không phản cảm thì sẽ có nhãn 0
insincere_question = train_df[train_df['target'] == 1].question_text #những câu hỏi không phản cảm thì sẽ có nhãn 1

* Trực quan hóa dữ liệu

In [ ]:
sns.countplot(data=train_df, x='target')

In [ ]:
 print("Insincere questions: ", insincere_question.shape[0] / train_df.shape[0])
 print("Sincere questions: ", sincere_question.shape[0] / train_df.shape[0])

Có thể thấy, dữ liệu giữa 2 lớp bị mất cân bằng lớn (nhãn 0 chiếm 93,81% còn nhãn 1 chiếm 6.19%). Việc mất cân bằng dữ liệu sẽ gây ra khó khăn trong việc dự đoán lớp thiểu số. Vì vậy, ngoài việc huấn luyện mô hình, em sẽ giải quyết thêm vấn đề mất cân bằng dữ liệu để đạt được kết quả tốt hơn với phương pháp undersampling với tỉ lệ lớp 1:lớp 0 = 1:4.

In [ ]:
sincere = train_df[train_df['target'] == 0]#những câu hỏi không phản cảm thì sẽ có nhãn 0
insincere = train_df[train_df['target'] == 1] #những câu hỏi không phản cảm thì sẽ có nhãn 1

In [ ]:
from sklearn.utils import resample
df_train_sampled = pd.concat([resample(sincere, replace = True, n_samples = len(insincere)*5), insincere])
df_train_sampled #dữ liệu đã được xử lý mất cân bằng

In [ ]:
sns.countplot(data=df_train_sampled, x='target')

Một số câu hỏi phản cảm trong tập dữ liệu:

In [ ]:
insincere_question.sample(n=5, random_state=4).values

Một số câu hỏi không phản cảm trong tập dữ liệu:

In [ ]:
sincere_question.sample(n=5, random_state=4).values

2.2. Tiền xử lý

Tiền xử lý dữ liệu bao gồm các việc: loại bỏ từ dừng, loại bỏ số, loại bỏ dấu câu, chuyển từ về dạng rút gọn.

In [ ]:
contraction_dict = {"ll": "will", "dont": "do not", "aint": "is not", "isnt": "is not", "doesnt": "does not"
, "cant": "cannot", "mustnt": "must not", "hasnt": "has not"
, "havent": "have not", "arent": "are not", "ain't": "is not", "aren't": "are not"
,"can't": "cannot", "‘cause": "because", "could've": "could have"
, "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not"
, "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would"
,"he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you"
, "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have"
, "I'll": "I will", "I'll've": "I will have","I'm": "I am", "Iam": "I am", "I've": "I have"
, "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have"
,"i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have"
, "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us"
, "ma'am": "madam", "mayn't": "may not", "might've": "might have"
,"mightn't": "might not","mightn't've": "might not have", "must've": "must have"
, "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not"
, "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not"
, "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not"
, "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have"
, "she'll": "she will", "she'll've": "she will have", "she's": "she is", 
"should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have"
, "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", 
"that'd've": "that would have", "that's": "that is", "there'd": "there would", 
"there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would"
, "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
"they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
"we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
"we'll've": "we will have", "we're": "we are", "we've": "we have", 
"weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
"what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", 
"who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
"why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", 
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
"y'all've": "you all have","you'd": "you would", "you'd've": "you would have", 
"you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [ ]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
nltk_stopwords = stopwords.words('english')
def preprocessing(text):
    # Data cleaning:
    
    text = re.sub('[0-9]{5,}','#####', text);
    text = re.sub('[0-9]{4,}','####', text);
    text = re.sub('[0-9]{3,}','###', text);
    text = re.sub('[0-9]{2,}','##', text); 
    text = re.sub('[0-9]{1,}','#', text); 
    text = re.sub(re.compile('<.*?>'), '', text)
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = text.lower()

    tokens = word_tokenize(text)
    tokens = [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in tokens]
    tokens = [w for w in tokens if w not in nltk_stopwords]
    tokens = [stemmer.stem(token) for token in tokens]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    # nối lại các từ vào chuỗi sau khi xử lý
    text = ' '.join(tokens) 

    return text


Thực hiện công việc tiền xử lý với tập dữ liệu train trước khi xử lý mất cân bằng:

In [ ]:
X_clean = []
for word in train_df.question_text:
  X_clean.append(preprocessing(word))

Thực hiện công việc tiền xử lý với tập dữ liệu train sau khi xử lý mất cân bằng:

In [ ]:
Y_clean = []
for word in df_train_sampled.question_text:
  Y_clean.append(preprocessing(word))

Thêm trường câu hỏi đã được tiền xử lý đối với cả 2 tập dữ liệu:

In [ ]:
train_df['cleaned_questions'] = X_clean
train_df.to_csv('output_preprocessed.csv', index=False)

In [ ]:
df_train_sampled['cleaned_questions'] = Y_clean
df_train_sampled.to_csv('Output_preprocessed.csv', index=False)

In [ ]:
pre_data_before = pd.read_csv('output_preprocessed.csv')

In [ ]:
preprocessed_data = pd.read_csv('Output_preprocessed.csv')

2 tập dữ liệu đó có dạng sau:

In [ ]:
train_df 

In [ ]:
df_train_sampled

**3. Huấn luyện mô hình**

Mô hình sử dụng: Logistic Regression và Linear SVM. Ngoài ra, em có thử sử dụng mô hình Random Forest nhưng thời gian chạy khá lâu so với các mô hình học máy cơ bản khác.
Phương pháp vector hóa dữ liệu: CountVectorizer và TF-IDFVectorizer (n-gram range: (1,2))
Chia dữ liệu để huấn luyện mô hình và dự đoán kết quả theo tỉ lệ test data:train data = 2:8. 
Em sẽ train với cả 2 bộ dữ liệu: trước khi xử lý mất cân bằng và sau khi xử lý mất cân bằng (tương đương với 8 lần train mô hình).

In [ ]:
from sklearn.model_selection import train_test_split

Tập train, test trước khi xử lý mất cân bằng:

In [ ]:
x, xtest, y, ytest = train_test_split(train_df['cleaned_questions'], train_df['target'], test_size = 0.2)

Tập train, test sau khi xử lý mất cân bằng:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train_sampled['cleaned_questions'], df_train_sampled['target'], test_size = 0.2)

**Mô hình Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,2))
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))


Train bằng phương pháp CountVectorizer với tập dữ liệu sau xử lý mất cân bằng:

In [ ]:
model = LogisticRegression(C=1, random_state=0)
vectorize_model_pipeline = Pipeline([
    ('count_vectorizer', count_vectorizer),
    ('model', model),
])

In [ ]:
vectorize_model_pipeline.fit(X_train, y_train)

In [ ]:
y_pred = vectorize_model_pipeline.predict(X_test)

In [ ]:
print('Accuracy :', accuracy_score(y_test, y_pred))
print('F1 score :', f1_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

Train bằng phương pháp TF_IDFVectorizer với tập dữ liệu sau xử lý mất cân bằng:

In [ ]:
vectorize_model_pipeline_ = Pipeline([
    ('tfidf_vectorizer', tfidf_vectorizer),
    ('model', model),
])

In [ ]:
vectorize_model_pipeline_.fit(X_train, y_train)

In [ ]:
y_pred_ = vectorize_model_pipeline_.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_))

Train bằng phương pháp CountVectorizer với tập dữ liệu trước xử lý mất cân bằng:

In [ ]:
vectorize_model_pipeline.fit(x, y)

In [ ]:
y_pred = vectorize_model_pipeline.predict(xtest)
print('Accuracy :', accuracy_score(ytest, y_pred))
print('F1 score :', f1_score(ytest, y_pred))
print(classification_report(ytest, y_pred))

Train bằng phương pháp TF_IDFVectorizer với tập dữ liệu trước xử lý mất cân bằng:

In [ ]:
vectorize_model_pipeline_.fit(x, y)

In [ ]:
y_pred = vectorize_model_pipeline_.predict(xtest)
print('Accuracy :', accuracy_score(ytest, y_pred))
print('F1 score :', f1_score(ytest, y_pred))
print(classification_report(ytest, y_pred))

Có thể thấy rằng, độ đo F1 của mô hình (nhãn 1) với tập dữ liệu sau khi xử lý mất cân bằng tăng khá nhiều:
CountVectorizer: 0.73 so với 0.53, TF_IDFVectorizer: 0.74 so với 0.54

**Mô hình Linear SVM**

Train bằng phương pháp CounerVectorizer với tập dữ liệu sau xử lý mất cân bằng:

In [ ]:
from sklearn.svm import LinearSVC
model1 = LinearSVC(random_state=3, tol=0.01, loss='hinge', C=1, verbose=2)
vectorize_model_pipeline1 = Pipeline([
    ('count_vectorizer', count_vectorizer),
    ('model1', model1),
])
vectorize_model_pipeline1.fit(X_train, y_train)

In [ ]:
y_pred1 = vectorize_model_pipeline1.predict(X_test)
print('Accuracy :', accuracy_score(y_test, y_pred1))
print('F1 score :', f1_score(y_test, y_pred1))
print(classification_report(y_test, y_pred1))

Train bằng phương pháp TF_IDFVectorizer với tập dữ liệu sau xử lý mất cân bằng:

In [ ]:
vectorize_model_pipeline1_ = Pipeline([
    ('count_vectorizer', tfidf_vectorizer),
    ('model1', model1),
])
vectorize_model_pipeline1_.fit(X_train, y_train)

In [ ]:
y_pred1 = vectorize_model_pipeline1_.predict(X_test)
print('Accuracy :', accuracy_score(y_test, y_pred1))
print('F1 score :', f1_score(y_test, y_pred1))
print(classification_report(y_test, y_pred1))

Train bằng phương pháp CountVectorizer với tập dữ liệu trước xử lý mất cân bằng:

In [ ]:
vectorize_model_pipeline1.fit(x, y)
y_pred1 = vectorize_model_pipeline1.predict(xtest)
print('Accuracy :', accuracy_score(ytest, y_pred1))
print('F1 score :', f1_score(ytest, y_pred1))
print(classification_report(ytest, y_pred1))

Train bằng phương pháp TF-IDFVectorizer với tập dữ liệu trước xử lý mất cân bằng:

In [ ]:
vectorize_model_pipeline1_.fit(x, y)
y_pred1 = vectorize_model_pipeline1_.predict(xtest)
print('Accuracy :', accuracy_score(ytest, y_pred1))
print('F1 score :', f1_score(ytest, y_pred1))
print(classification_report(ytest, y_pred1))

Có thể thấy rằng, độ đo F1 của mô hình (nhãn 1) với tập dữ liệu sau khi xử lý mất cân bằng tăng khá nhiều:
CountVectorizer: 0.74 so với 0.54, TF_IDFVectorizer: 0.76 so với 0.56

In [ ]:
# from sklearn.metrics import classification_report
# print(classification_report(y_test, predictions1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
count_vectorizer = CountVectorizer()
model2 = RandomForestClassifier()

vectorize_model_pipeline2 = Pipeline([
    ('count_vectorizer', count_vectorizer),
    ('model2', model2)])
vectorize_model_pipeline2.fit(X_train, y_train)
predictions2 = vectorize_model_pipeline2.predict(X_test)

print('Accuracy :', accuracy_score(y_test, predictions2))
print('F1 score :', accuracy_score(y_test, predictions2))

In [ ]:
print(classification_report(y_test, predictions2))

**Nộp kết quả**

Các lần submit cho thấy mô hình Logistic Regession với CountVectorizer cho kết quả tốt nhất.

In [ ]:
test_df['preprocessing'] = test_df['question_text'].apply(preprocessing) #tiền xử lý với dữ liệu test

In [ ]:
predictions = vectorize_model_pipeline.predict(test_df['preprocessing']) #dự đoán kết quả

In [ ]:
test_df['prediction'] = predictions
results = test_df[['qid', 'prediction']]
results.to_csv('submission.csv', index=False) #lưu vào file submission

In [ ]:
results.head()